### Import

In [3]:
import requests
import pymysql
import json
import pandas as pd
import datetime as dt
import pickle
import time
from tqdm import tqdm
from bs4 import BeautifulSoup as bs

### DB 사용 예시

In [44]:
con = pymysql.connect(host='localhost', user='ssafy', password='ssafy', 
                      db='access_db', charset='utf8mb4', autocommit=True)
cur = con.cursor()

sql = "INSERT INTO table_name (value1, value2) VALUES (123, '이름');"
cur.execute(sql)

con.commit()
con.close()


### 환율

#### api 함수

In [3]:
def exchange_rate_api(date):
  API_HOST = "https://www.koreaexim.go.kr/site/program/financial/exchangeJSON"
  key = "?authkey=b60MyMtLUDVQyyiDjZRdFGbKxoJBypkl"
  search = "&searchdate="+date
  data = "&data=AP01"
  
  url = API_HOST+key+search+data
  headers = {"Content-Type" : "application/json", "charset" : "UTF-8", "Accept":"*/*"}

  try:
    response = requests.get(url, headers=headers)
    return response
  except Exception as ex:
    print(ex)
  

#### 데이터 수집

In [5]:
date_range = pd.date_range(start='11/1/2014', end='12/31/2022')

result_list = []
for date in date_range:
  if dt.date.weekday(date) in (5, 6) :
    continue
  str_date = str(date).split(" ")[0]
  
  r = json.loads(exchange_rate_api(str_date).text)
  
  flag = True
  for data in r:
    if data["result"] == 4:
      flag = False
      break
    if data["cur_nm"] in ('미국 달러', '일본 옌', '유로') :
      if(data["cur_nm"] == '일본 옌'):
        result_list.append("('{}', '{}', {})".format(str_date, '일본 100엔', data["deal_bas_r"].replace(",", "")))
      else:
        result_list.append("('{}', '{}', {})".format(str_date, data['cur_nm'], data["deal_bas_r"].replace(",", "")))
  if flag == False:
    break

#### 저장 및 불러오기

In [6]:
with open("data(cur_).p", "wb") as f:
  pickle.dump(result_list, f)

In [5]:
with open("data.p", "rb") as f:
  result_list = pickle.load(f)

#### DB 저장

In [8]:
con = pymysql.connect(host='localhost', user='ssafy', password='ssafy', 
                      db='access_db', charset='utf8mb4', autocommit=True)
cur = con.cursor()

In [10]:
sql = "INSERT INTO currency(cur_date, cur_name, cur_rate) VALUES " + ",".join(result_list)
cur.execute(sql)
con.close()

### 원자재(석유, 금)

- API가 2020년부터 데이터가 있어서 사용이 불가능.. 하하
- Naver를 통해 크롤링해야할 듯.
  - 주식, 코스닥 코스피도 동일
- 석유 : 경유, 휘발유, 등유 3가지 평균값 활용
  - 네이버 증권에는 등유가 없어서 경유, 휘발유만 사용?
- 금 : 국내금, 국제금 2가지 각각 활용

#### api 함수

##### 오일
- 휘발유 : OIL_GSL
- 경유 : OIL_LO
- 예시 : https://finance.naver.com/marketindex/oilDailyQuote.naver?marketindexCd=OIL_GSL&page=640

In [ ]:
def oil_api(oil_type):
  page = 641
  end = 15
  oil = "휘발유" if oil_type == "OIL_GSL" else "경유"
  HOST = "https://finance.naver.com/marketindex/oilDailyQuote.naver?marketindexCd={}&page=".format(oil_type)
  
  headers = {"Content-Type" : "application/json", "charset" : "UTF-8", "Accept":"*/*"}

  try:
    result_list = []
    for p in tqdm(range(page, end-1, -1), 
              total = page-end+1, ## 전체 진행수
              desc = 'Desc', ## 진행률 앞쪽 출력 문장
              ncols = 80, ## 진행률 출력 폭 조절
              leave = True, ## True 반복문 완료시 진행률 출력 남김. False 남기지 않음.
            ):
      url = HOST+str(p)
      response = requests.get(url, headers=headers)
      result_list.extend(oil_bs(response, oil))
      
      time.sleep(0.5)
      
    return result_list
  except Exception as ex:
    print("oil_api 오류 발생")
    print(ex)

def oil_bs(response, oil_type):
  start = dt.datetime.strptime("2011-01-01", "%Y-%m-%d")
  end = dt.datetime.strptime("2023-01-01", "%Y-%m-%d")
  
  soup = bs(response.text, "html.parser")
  result = soup.select("tbody > tr")
  
  result_list = []  
  try:
    for p in result[::-1]:
      state = p.get("class")[0]
      td_list = p.select("td")
      td = [td_list[0].getText().strip().replace(".", "-"), oil_type, state, td_list[1].getText().strip(), td_list[2].getText().strip(), td_list[3].getText().strip()[1:-1].strip()]
      
      cur_date = dt.datetime.strptime(td[0], "%Y-%m-%d")
      start_diff = (cur_date - start).total_seconds()
      end_diff= (end - cur_date).total_seconds()
      if start_diff < 0 or end_diff <= 0:
        continue
      
      result_list.append("('{}', '{}', '{}', {}, {}, {})".format(td[0], td[1], td[2], td[3].replace(",", ""), td[4], td[5]))
  except Exception as ex:
    print("oil_bs 오류 발생")
    print(ex)
  
  return result_list

##### 금
- 국제금
  - 예시 : https://finance.naver.com/marketindex/worldDailyQuote.naver?marketindexCd=CMDT_GC&fdtc=2&page=446
- 국내금
  - 예시 : https://finance.naver.com/marketindex/goldDailyQuote.naver?&page=305



In [49]:
def gold_api(gold_type):
  if gold_type == "national":
    HOST = "https://finance.naver.com/marketindex/worldDailyQuote.naver?marketindexCd=CMDT_GC&fdtc=2&page="
    page = 446
    end = 10
    gold = "국제금(달러/트로이온스)"
  else :
    HOST = "https://finance.naver.com/marketindex/goldDailyQuote.naver?&page="
    page = 305
    end = 8
    gold = "국내금(원/g)"
  
  headers = {"Content-Type" : "application/json", "charset" : "UTF-8", "Accept":"*/*"}

  try:
    result_list = []
    for p in tqdm(range(page, end-1, -1), 
              total = page-end+1, ## 전체 진행수
              desc = 'Desc', ## 진행률 앞쪽 출력 문장
              ncols = 80, ## 진행률 출력 폭 조절
              leave = True, ## True 반복문 완료시 진행률 출력 남김. False 남기지 않음.
            ):
      url = HOST+str(p)
      response = requests.get(url, headers=headers)
      result_list.extend(gold_bs(response, gold))
      
      time.sleep(0.5)
      
    return result_list
  except Exception as ex:
    print("gold_api 오류 발생")
    print(ex)

def gold_bs(response, gold_type):
  start = dt.datetime.strptime("2011-01-01", "%Y-%m-%d")
  end = dt.datetime.strptime("2023-01-01", "%Y-%m-%d")
  
  soup = bs(response.text, "html.parser")
  result = soup.select("tbody > tr")
  
  result_list = []  
  try:
    for p in result[::-1]:
      state = p.get("class")[0]
      td_list = p.select("td")
      
      td = [td_list[0].getText().strip().replace(".", "-"), gold_type, state, td_list[1].getText().strip(), td_list[2].getText().strip()]
      if gold_type == "국제금(달러/트로이온스)" :
        td.append(td_list[3].getText().strip()[1:-1].strip())
      else :
        temp_value = float(td[-2]) + float(td[-1]) if td[2] == 'down' else (-float(td[-1]))
        td.append(str(round(float(td[-1]) / temp_value, 1)))
      
      cur_date = dt.datetime.strptime(td[0], "%Y-%m-%d")
      start_diff = (cur_date - start).total_seconds()
      end_diff= (end - cur_date).total_seconds()
      if start_diff < 0 or end_diff <= 0:
        continue
      
      result_list.append("('{}', '{}', '{}', {}, {}, {})".format(td[0], td[1], td[2], td[3].replace(",", ""), td[4], td[5]))
  except Exception as ex:
    print("gold_bs 오류 발생")
    print(ex)
  
  return result_list

#### 데이터 수집

##### 오일

In [50]:
result_list = gold_api("national")

Desc: 100%|███████████████████████████████████| 437/437 [04:29<00:00,  1.62it/s]


In [53]:
result_list = gold_api("daily")

Desc:   0%|                                             | 0/298 [00:00<?, ?it/s]

gold_bs 오류 발생
could not convert string to float: '51,445.67'


Desc:   0%|                                     | 1/298 [00:00<02:56,  1.69it/s]

gold_bs 오류 발생
could not convert string to float: '50,783.09'


Desc:   1%|▏                                    | 2/298 [00:01<03:01,  1.63it/s]

...

gold_bs 오류 발생
could not convert string to float: '48,056.67'


Desc:   1%|▍                                    | 4/298 [00:02<03:05,  1.59it/s]

gold_bs 오류 발생
could not convert string to float: '50,624.57'


Desc:   1%|▍                                    | 4/298 [00:03<03:48,  1.29it/s]


KeyboardInterrupt: 

#### 저장 및 불러오기

In [52]:
with open("data(national_gold_230414).p", "wb") as f:
  pickle.dump(result_list, f)

In [14]:
with open("data(domestic_gold_230414).p", "rb") as f:
  result_list = pickle.load(f)

#### DB 저장

In [39]:
con = pymysql.connect(host='localhost', user='ssafy', password='ssafy', 
                      db='access_db', charset='utf8mb4', autocommit=True)
cur = con.cursor()

In [40]:
sql = "INSERT INTO material(material_date, material_name, material_state, material_rate, material_change, material_change_rate) VALUES " + ",".join(result_list)
cur.execute(sql)
con.close()